### Computer Vision Project 1: Detecting Of Aruco Markers And Placing Images Over Them

#### 1- Initializion of Libraries.

In [2]:
import os
import cv2
import numpy as np
import cv2.aruco as aruco

#### 2-Initializion of Directories and Parameters.

In [30]:
images_dir = "images"
output_dir = "informationimages"
final_output_dir = "finalimages"
overlay_img_path = "image.jpg" 

# List of dictionaries to try for detection
aruco_dicts = [
    cv2.aruco.DICT_4X4_50,
    cv2.aruco.DICT_4X4_100,
    cv2.aruco.DICT_4X4_250,
    cv2.aruco.DICT_4X4_1000,
    cv2.aruco.DICT_5X5_50,
    cv2.aruco.DICT_5X5_100,
    cv2.aruco.DICT_5X5_250,
    cv2.aruco.DICT_5X5_1000,
    cv2.aruco.DICT_6X6_50,
    cv2.aruco.DICT_6X6_100,
    cv2.aruco.DICT_6X6_250,
    cv2.aruco.DICT_6X6_1000,
    cv2.aruco.DICT_7X7_50,
    cv2.aruco.DICT_7X7_100,
    cv2.aruco.DICT_7X7_250,
    cv2.aruco.DICT_7X7_1000,
    cv2.aruco.DICT_ARUCO_ORIGINAL,
    cv2.aruco.DICT_APRILTAG_16h5,
    cv2.aruco.DICT_APRILTAG_25h9,
    cv2.aruco.DICT_APRILTAG_36h10,
    cv2.aruco.DICT_APRILTAG_36h11,
    cv2.aruco.DICT_ARUCO_MIP_36h12
]
#initialize the output paths if they dont exist:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(final_output_dir):
    os.makedirs(final_output_dir)


#### 3- Main algortihm, containing detection, marking and overlaying of the image.

In [31]:
# Read the overlay image
overlay_img = cv2.imread(overlay_img_path)

# Iterate over each image
for img_name in os.listdir(images_dir):
    img_path = os.path.join(images_dir, img_name)
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flag to track if markers are found
    markers_found = False
    
    # Iterate over each dictionary for detection
    for aruco_dict_id in aruco_dicts:
        aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_id)
        #initialize the ArUco Parameters.
        parameters = cv2.aruco.DetectorParameters()
        detector = cv2.aruco.ArucoDetector(aruco_dict,parameters)
        # Detect markers using the current dictionary
        corners, ids, _ = detector.detectMarkers(gray)
        
        # If markers are found, proceed with further processing
        if ids is not None:
            markers_found = True
            break  # Exit the loop if markers are found
        
    if markers_found:
        for j in range(len(ids)):
            # Calculate the center coordinates, other coordinates, and angles
            c = corners[j][0]
            x_center = int(c[:, 0].mean())
            y_center = int(c[:, 1].mean())
            x_min, y_min = np.min(c, axis=0)
            x_max, y_max = np.max(c, axis=0)
            angle = cv2.minAreaRect(c)[-1]

            # Write these corner, center, and angle values on the top left corner of the image
            cv2.putText(img, f"ID: {img_name}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.putText(img, f"Center: ({x_center}, {y_center})", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.putText(img, f"Box: ({x_min}, {y_min}), ({x_max}, {y_min}), ({x_max}, {y_max}), ({x_min}, {y_max})", 
                        (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.putText(img, f"Angle: {angle:.2f} degrees", (10, 260), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            img_with_markers = cv2.aruco.drawDetectedMarkers(img.copy(), corners, ids, borderColor=(0, 255, 0))
            cv2.imwrite(os.path.join(output_dir, f"marked_{img_name}"), img_with_markers)

            # Get the overlay image, resize it and put it over the marker using the perspective operations
            overlay_img_resized = cv2.resize(overlay_img, (int(np.linalg.norm(c[0] - c[1])), int(np.linalg.norm(c[0] - c[3]))))
            pts_dst = np.array([[0, 0],                                           #Top-left corner
            [overlay_img_resized.shape[1] - 1, 0],                                #Top-right corner
            [overlay_img_resized.shape[1] - 1, overlay_img_resized.shape[0] - 1], #Bottom-right corner
            [0, overlay_img_resized.shape[0] - 1]], dtype=np.float32)             #Bottom-left corner

            perspective_matrix, _ = cv2.findHomography(pts_dst, c)
            overlay_img_warped = cv2.warpPerspective(overlay_img_resized, perspective_matrix, (img.shape[1], img.shape[0]))
            
            # Create, invert and convert the mask of overlay and the original image
            mask = np.zeros_like(img)
            cv2.fillPoly(mask, [np.int32(c)], (255, 255, 255))
            mask_inv = cv2.bitwise_not(mask)
            img_with_overlay = cv2.bitwise_and(img, mask_inv)
            img_with_overlay += overlay_img_warped
            
            # Save the final image
            cv2.imwrite(os.path.join(final_output_dir, f"overlay_{img_name}"), img_with_overlay)    
    else:
        # Annotate the fact that there are no markers found
        cv2.putText(img, "No markers found.", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        # Save the final image
        cv2.imwrite(os.path.join(output_dir, f"marked_{img_name}"), img)